In [ ]:
#import packages
import billboard
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

#init list to store chart songs
chart_lists = list()

# list of billboard charts to grab
chart_titles=['pop-songs', 'alternative-songs','hot-country-songs','Hot-Rap-Songs','Hot-100-Songs']

#connect to spotify
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="cfe66cda140c43348dd2579cd81a0e84",client_secret="40c03ac11243418a96dcfb0fb5c09a39"))

#loop through billboard charts
for chart_title in chart_titles:
    #init list for songs for this chart
    song_list = list()
    # loop through 10 years of billboard charts
    for year in range(2021,2011,-1):
        #get chart for a year
        chart = billboard.ChartData(chart_title, year=year)
        #print 1st entry
        print(chart_title, year, "1st:", chart[0].title)
        #loop through songs
        for song in chart:
            #get spotify song information
            result = spotify.search(q=f"track:{song.title}", type="track", market="US")
            #get song's dictionary
            test=result["tracks"]["items"]
            #if it foud the song, add it to the song list
            if len(test) !=0:
                #print(test[0])
                song_list.append(test[0])
    #append song list to chart list              
    chart_lists.append(song_list)



pop-songs 2021 1st: Levitating


In [108]:
#list to store the chart dataframes
df_list = list()
#loop through charts
for charti in chart_lists:
    #convert dictionary to dataframe
    df = pd.DataFrame.from_dict(charti)
    #add some columns for album info
    album= df['album'].to_list()
    df_album= pd.DataFrame.from_dict(album)
    df['album_type'] = df_album['album_type']
    df['album_name'] = df_album['name']
    df['release_year'] = df_album['release_date'].str[:4]
    
    #add to list of datframes
    df_list.append(df.copy())
    
print(len(df_list))

5


In [109]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="cfe66cda140c43348dd2579cd81a0e84",client_secret="40c03ac11243418a96dcfb0fb5c09a39"))

#list to store features dataframes
df_list2 = list()
# loop through list of dataframes
for k in range(len(df_list)):
    # get the spotify song ids
    billboard_ids = df_list[k]["id"].tolist()
    #number of records to pull
    n = 100
    #list to store features
    list_feat = list()
    #print chart titles
    print(chart_titles[k])
    
    #loop through songs in dataframe
    for i in range(0,len(billboard_ids),n):
        #get there features
        result_feat = spotify.audio_features(billboard_ids[i:i+n])
        
        #loop throug songs features
        for j in range(len(result_feat)):
            #if found, add to list _feature
            if str(result_feat[j] ) != "None":
                list_feat.append(result_feat[j])
    #convert dictionary to dataframe
    df_billboard_feat = pd.DataFrame.from_dict(list_feat) 
    #print length
    print(len(df_billboard_feat)) 
    
    #merge dataframes on id
    df = pd.merge(df_list[k], df_billboard_feat, on=["id"])
    
    #drop duplicates
    df.drop_duplicates(subset="id",
                     keep=False, inplace=True)
    
    #drop unneeded columns
    df.drop(['album','artists','external_ids', 'external_urls', 'href', 'preview_url','uri_x', 'uri_y', 'track_href', 'analysis_url', 'duration_ms_y','type_y'], axis=1, inplace=True)
    
    #add to list of dataframes for different charts
    df_list2.append(df.copy())

print(len(df_list2))
df_list2[0].columns

pop-songs
499
alternative-songs
496
hot-country-songs
996
Hot-Rap-Songs
499
Hot-100-Songs
999
5


Index(['disc_number', 'duration_ms_x', 'explicit', 'id', 'is_local',
       'is_playable', 'name', 'popularity', 'track_number', 'type_x',
       'album_type', 'album_name', 'release_year', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [110]:
# add a column for Billboard Genre Chart and save as CSV file
df_list2[0]["Genre"] = "Pop"
print(len(df_list[0]))
print(len(df_list2[0]))
df_list2[0].to_csv("pop-songs-billboard.csv")
df_list2[1]["Genre"] = "Alternative"
df_list2[1].to_csv("alt-songs-billboard.csv")
df_list2[2]["Genre"] = "Country"
df_list2[2].to_csv("country-songs-billboard.csv")
df_list2[3]["Genre"] = "Rap"
df_list2[3].to_csv("rap-songs-billboard.csv")
df_list2[4]["Genre"] = "Hot 100"
df_list2[4].to_csv("hot100-songs-billboard.csv")
df_list2[0].columns

499
407


Index(['disc_number', 'duration_ms_x', 'explicit', 'id', 'is_local',
       'is_playable', 'name', 'popularity', 'track_number', 'type_x',
       'album_type', 'album_name', 'release_year', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'Genre'],
      dtype='object')